<a href="https://colab.research.google.com/github/BrianThomasRoss/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel, ttest_1samp


In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-10-07 20:02:28--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-10-07 20:02:28 (286 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
# Load Data
df = pd.read_csv('house-votes-84.data', 
                 header=None,
                 names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

In [0]:
df = df.replace({'?': np.NaN, 'y':1, 'n':0})



In [0]:
# Filtering

republicans = df[df.party == "republican"]
democrats = df[df.party == "democrat"]

In [0]:

issues_list = df.columns.to_list()
issues_list.remove('party')

In [0]:
# Define a function to compare the means between parties

def compare_means(issues):
  BOLD = '\033[1m'
  END = '\033[0m'
  UNDERLINE = '\033[4m'

  d = BOLD + "Dem mean is: " + END
  r = BOLD + "Rep mean is: " + END
 
  for issue in issues: 
    dem_mean = str(np.mean(democrats[issue]))
    rep_mean = str(np.mean(republicans[issue]))

    issue_str = UNDERLINE + str(issue).title().replace("-", " ") + END

    print("\tIssue: " + issue_str + 
          "\n" + d + dem_mean +
          "\n" + r + rep_mean)
    print("\n" + "-"*40)
    

In [215]:
compare_means(issues_list)

	Issue: Handicapped Infants
Dem mean is: 0.6046511627906976
Rep mean is: 0.18787878787878787

----------------------------------------
	Issue: Water Project
Dem mean is: 0.502092050209205
Rep mean is: 0.5067567567567568

----------------------------------------
	Issue: Budget
Dem mean is: 0.8884615384615384
Rep mean is: 0.13414634146341464

----------------------------------------
	Issue: Physician Fee Freeze
Dem mean is: 0.05405405405405406
Rep mean is: 0.9878787878787879

----------------------------------------
	Issue: El Salvador Aid
Dem mean is: 0.21568627450980393
Rep mean is: 0.9515151515151515

----------------------------------------
	Issue: Religious Groups
Dem mean is: 0.47674418604651164
Rep mean is: 0.8975903614457831

----------------------------------------
	Issue: Anti Satellite Ban
Dem mean is: 0.7722007722007722
Rep mean is: 0.24074074074074073

----------------------------------------
	Issue: Aid To Contras
Dem mean is: 0.8288973384030418
Rep mean is: 0.1528662420382

In [147]:
# Find an issue that republicans support more than democrats

ttest_ind(republicans['physician-fee-freeze'], 
          democrats['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [149]:
# Find an issue that democrats support more than republicans

ttest_ind(republicans['aid-to-contras'], 
          democrats['aid-to-contras'], nan_policy='omit')

Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)

In [160]:
## Using hypothesis testing, find an issue where the difference between republicans
## and democrats has p > 0.1 (i.e. there may not be much of a difference)

ttest_ind(republicans['water-project'],
          democrats['water-project'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [0]:
def two_sample_ttest(issues):
  BOLD = '\033[1m'
  END = '\033[0m'

  for issue in issues:
    ttest_result = str(ttest_ind(republicans[issue],
                                 democrats[issue],
                                 nan_policy='omit'))
    
    print(BOLD + "Issue: " + 
          str(issue).title().replace("-", " ") + END + "\n" +
          ttest_result + "\n" + "----"*20)
    

In [205]:
two_sample_ttest(issues_list)

Issue: Handicapped Infants
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
--------------------------------------------------------------------------------
Issue: Water Project
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
--------------------------------------------------------------------------------
Issue: Budget
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
--------------------------------------------------------------------------------
Issue: Physician Fee Freeze
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
--------------------------------------------------------------------------------
Issue: El Salvador Aid
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
--------------------------------------------------------------------------------
Issue: Religious Groups
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
----